In [1]:
import numpy as np
import sys
import os
sys.path.append(os.path.join(os.getcwd(),"../Algorithm encapsulation"))
sys.path.append(os.path.join(os.getcwd(),".."))
print(sys.path)
from Multiband import Multiband
from MPbandTool import MPbandTool
import pandas as pd
from ga_platoon import max_dt

['f:\\workspace\\jupyter\\论文\\graduation thesis\\Case Analysis', 'd:\\anaconda\\envs\\cplex\\python37.zip', 'd:\\anaconda\\envs\\cplex\\DLLs', 'd:\\anaconda\\envs\\cplex\\lib', 'd:\\anaconda\\envs\\cplex', '', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\chardet-3.0.4-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\docloud-1.0.375-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\urllib3-1.25.11-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\fonttools-4.36.0-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\cvxopt-1.3.0-py3.7-win-amd64.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\pytz-2022.2.1-py3.7.egg', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\win32\\lib', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\Pythonwin', 'd:\\anaconda\\envs\\cplex\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\0\\.ipython', 'f:\\wor

In [2]:
def get_rf(d,p):
    tmp=[]
    for i,a in enumerate(p):
        idx=np.where(a!=0)[0]
        a=a&0
        if len(idx)>0:
            a[:idx[0]]=1
        tmp.append((d[i]*a).sum())
    return tmp

In [3]:
phase = np.loadtxt("./data/phase")
cir=phase[0].sum()
phase=phase/cir
vol = np.loadtxt("./data/volume")
qv = np.loadtxt("./data/path_vol")
phase_get = np.load("./data/phase_get.npy")
d = np.loadtxt("./data/distance")
straight_get = np.load("./data/straight_get.npy")
ison = np.array([0, 0, 0, 1, 1, 1])
px = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
pxb = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
               [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
qb=[120,120]
lower, upper = 8,14
linspace = np.arange(lower, upper, 1)
lin_num = len(linspace)

# 南北直行方向绿灯时间
sg=np.array([(straight_get[i]*phase).sum(axis=1) for i in range(len(straight_get))])
rho = vol[0] / vol[1]
num = len(vol[0])
numr = len(qv)
nump = len(phase_get)
M = 1e6
nx = 0.00000001   
be = 8
speedScope = np.array([lower, upper])
speedVar = np.array([-2, 2])

g=np.array([(phase_get[i]*phase).sum(axis=1) for i in range(numr) ])
r=1-g 
rf=np.array([get_rf(phase,phase_get[i]) for i in range(numr)])

ex=[1,0,1,0,1,0,1,1,1,1,1,0]
dwt=15
qb_x=np.ones_like(sg)*20
cap=np.ones_like(sg,dtype=int)*3

spd_on=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","上行速度数据.csv"),index_col=0)
spd_in=pd.read_csv(os.path.join(os.getcwd(),"..","speed analysis","下行速度数据.csv"),index_col=0)

p=[0,0,0,0,1,0,0,0,1,0,1,0,0]

In [4]:
mb=Multiband(phase,[80,180],straight_get,p,d,lower,upper,speedVar[0], speedVar[1],vol,px)
mb._solve()

13 13
conflict(s): 0
status  = integer optimal solution
time    = 0.031 s.
problem = MILP
gap     = 0%

object value: 2111.0150000000003


In [5]:
a=mb.get_dataframe()
a

,cross_number,distance,sg1,sg2,b1,b2,offset,p,t1,t2,z,u1,u2,w1,car_t1,w2,car_t2
0,1.0,800.0,0.282,0.380,0.282,0.380,0.000000,0.0,91.520000,68.960000,80.000000,0.000000,0.000000,0.1410,11.139000,0.1900,37.570000
1,2.0,520.0,0.426,0.368,0.260,0.368,80.000000,0.0,37.142857,49.257143,80.000000,0.000000,0.000000,0.2850,102.670000,0.1840,128.616000
2,3.0,500.0,0.260,0.230,0.260,0.230,49.542857,0.0,38.980000,62.500000,80.000000,0.000000,0.000000,0.1300,59.812857,0.1150,79.427857
3,4.0,490.0,0.277,0.277,0.277,0.230,7.842857,0.0,61.250000,61.250000,80.000000,0.000000,0.000000,0.1385,18.784357,0.1150,16.927857
4,5.0,370.0,0.255,0.255,0.255,0.255,0.000000,1.0,36.594881,46.250000,136.933687,0.125339,0.319911,0.1275,17.331545,0.1275,17.331545
5,6.0,254.0,0.650,0.650,0.243,0.255,36.594881,0.0,22.338806,31.750000,136.933687,0.000000,0.000000,0.1275,53.932426,0.5225,108.015232
6,7.0,585.0,0.243,0.243,0.165,0.243,59.755289,0.0,71.837984,65.095703,136.933687,0.000000,0.000000,0.1215,76.310232,0.1215,76.271232
7,8.0,1020.0,0.165,0.165,0.165,0.165,0.000000,0.0,127.500000,72.857143,136.933687,0.000000,0.000000,0.0825,11.214529,0.0825,11.214529
8,9.0,409.0,0.298,0.298,0.298,0.298,32.115000,1.0,51.125000,32.115000,80.000000,0.536830,0.000001,0.1490,43.886000,0.1490,43.886000
9,10.0,547.0,0.379,0.379,0.379,0.298,0.000000,0.0,68.375000,68.375000,80.000000,0.000000,0.000000,0.1895,14.970500,0.1490,11.771000


In [6]:
a[["b"+str(i+1) for i in range(2)]].mean()

b1    0.274077
b2    0.299308
dtype: float64

In [7]:
mpb=MPbandTool(phase, [80,180], vol, qv, phase_get, d, straight_get, ison, px, pxb, qb, lower, upper, 
            be, speedVar[0], speedVar[1],ex, dwt,lower,upper,speedVar[0], speedVar[1],p,a.offset/a.z,
            1/a.z,np.array([a.t1/a.z,a.t2/a.z]))
mpb._solve()

conflict(s): 0
Version identifier: 12.10.0.0 | 2019-11-26 | 843d4de2ae
CPXPARAM_Read_DataCheck                          1
CPXPARAM_RandomSeed                              201903125
1 of 1 MIP starts provided solutions.
MIP start 'm1' defined initial solution with objective 0.0000.
Tried aggregator 3 times.
MIP Presolve eliminated 767 rows and 262 columns.
MIP Presolve modified 946 coefficients.
Aggregator did 62 substitutions.
Reduced MIP has 450 rows, 220 columns, and 1136 nonzeros.
Reduced MIP has 35 binaries, 29 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (1.93 ticks)
Probing fixed 0 vars, tightened 52 bounds.
Probing time = 0.00 sec. (0.12 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 64 rows and 58 columns.
MIP Presolve modified 206 coefficients.
Reduced MIP has 386 rows, 162 columns, and 972 nonzeros.
Reduced MIP has 17 binaries, 29 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.64 ticks)
Probing time = 0.00 sec. (0.08 ticks)
Tried ag

In [8]:
# colors=[
#             {"color":"gray" , "hatch": None, "fill": True},
#             {"color":"gray", "hatch":"||||" , "fill": True},
#             {"color": "gray", "hatch":"////" , "fill": True},
#             {"color":"white", "hatch": "****", "fill": False},
#             {"color": "white", "hatch": None, "fill": False},
#             {"color": "white", "hatch": "////", "fill": False},
#             {"color": "white", "hatch": "||||", "fill": False},
#             {"color": "white", "hatch": "\\\\\\\\", "fill": False},
#             {"color": "white", "hatch": "xxxx", "fill": False},
#             {"color": "white", "hatch": "....", "fill": False},
# ]
# linestyles=[
#     {"linestyle":"-" },
#     {"linestyle":(5,(10,3))},
#     {"linestyle":(0, (5, 3)) },
#     {"linestyle":":" },
#     {"linestyle":(0, (3, 1, 1, 1)) },
#     {"linestyle": (0, (5, 10)) },
# ]
# legends=["1->15", "1->6", "8->15", "15->2", "14->9", "7->1"]
# mb.draw_car_bound("./时距图",colors,legends,2,linestyles)

In [9]:
from  SimAnalysis import SimAnalysis
df=mpb.get_dataframe()
df["dw1"]=np.ones_like(df.z)*15
df["dw2"]=np.ones_like(df.z)*15
sim=SimAnalysis(df,phase,ex,os.path.join(os.getcwd(),"..","vissim","aa","zhongsan.inp"),
               os.path.join(os.getcwd(),"..","vissim","aa","vissim.ini"),
               os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.inp"))
sim.start()
spdx=sim.analysis_speed(os.path.join(os.getcwd(),"..","vissim","aa","zhongsan - 副本.fzp"))
delayx=sim.analysis_delay(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.vlz'))
tvltx=sim.analysis_traveltime(os.path.join(os.getcwd(),"..","vissim","aa",'zhongsan - 副本.rsz'))

In [33]:
spdx.to_csv("./sensitivity/spdx/multiband.csv",encoding="utf-8-sig")

In [30]:
i=8
delayx[["delay"+str(i),"stoped"+str(i),"stops"+str(i),"vehicle"+str(i)]]

,delay8,stoped8,stops8,vehicle8
0,0.0,0.0,0.00,0
1,338.7,203.0,13.00,1
2,345.6,230.8,13.67,9
3,421.0,298.2,19.71,7
4,457.8,298.2,34.40,5
5,512.3,338.5,36.25,16
6,444.2,296.7,27.00,38


In [31]:
a=tvltx["traveltime"+str(i)].astype(float)
a[a!=0].mean()

1190.0249999999999

In [12]:
a=mpb.get_dataframe()
a

,cross_number,distance,b1,b2,b3,b4,b5,b6,offset,p,...,u1,u2,u3,u4,u5,u6,bb1,bb2,tb1,tb2
0,1.0,800.0,0.282,0.282,0.000,0.276343,0.000,0.380,0.000000,0.0,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.282,0.380,72.142857,72.142857
1,2.0,520.0,0.260,0.260,0.000,0.368000,0.000,0.368,80.000000,0.0,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.260,0.368,37.142857,37.142857
2,3.0,500.0,0.260,0.260,0.000,0.230000,0.000,0.230,49.542857,0.0,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.260,0.230,50.714286,50.714286
3,4.0,490.0,0.277,0.277,0.000,0.230000,0.000,0.230,7.842857,0.0,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.277,0.230,35.000000,35.000000
4,5.0,370.0,0.255,0.000,0.255,0.255000,0.000,0.255,0.000000,1.0,...,0.125339,1.000000e-08,0.263839,3.199107e-01,1.000000e-08,3.199107e-01,0.255,0.255,41.428571,41.428571
5,6.0,254.0,0.243,0.000,0.243,0.255000,0.000,0.255,36.594881,0.0,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.243,0.255,18.142857,18.142857
6,7.0,585.0,0.165,0.000,0.165,0.183188,0.000,0.243,59.755289,0.0,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.165,0.243,56.785714,56.785714
7,8.0,1020.0,0.165,0.000,0.165,0.165000,0.000,0.165,0.000000,0.0,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.165,0.165,87.857143,87.857143
8,9.0,409.0,0.298,0.000,0.298,0.298000,0.298,0.000,32.115000,1.0,...,0.536830,1.000000e-08,0.536830,1.000000e-08,8.250100e-02,1.000000e-08,0.298,0.298,44.214286,44.214286
9,10.0,547.0,0.379,0.000,0.379,0.298000,0.298,0.000,0.000000,0.0,...,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.379,0.298,54.071429,54.071429


In [13]:
a.b1.mean(),a.b2[:8].mean(),a.b3[4:].mean(),a.b4.mean(),a.b5[7:].mean(),a.b6[:8].mean(),a.bb1.mean(),a.bb2.mean()

(0.2740769230769231,
 0.13487500000000002,
 0.27600000001158986,
 0.2867331469359975,
 0.2941666666666667,
 0.2657500000025612,
 0.2740769230769231,
 0.2993076923076924)

In [14]:
a.y3

0     0.0
1     0.0
2     0.0
3     0.0
4     1.0
5     1.0
6     1.0
7     1.0
8     1.0
9     1.0
10    1.0
11    1.0
12    1.0
Name: y3, dtype: float64